# Lab in Data Science: Final Project

Pierre Fouche, Matthias Leroy and Raphaël Steinmann

## Imports

In [ ]:
%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = (10,6)
plt.rcParams['font.size'] = 18
plt.style.use('fivethirtyeight')

In [ ]:
import getpass
import pyspark
from datetime import datetime
from pyspark.sql import SparkSession
import pyspark.sql.functions as functions
import math
import helpers

%load_ext autoreload
%autoreload 2

## Initialize the `SparkSession`

In [ ]:
conf = pyspark.conf.SparkConf()
conf.setMaster('yarn')
conf.setAppName('project-{0}'.format(getpass.getuser()))
conf.set('spark.executor.memory', '16g')
conf.set('spark.executor.instances', '10')
conf.set('spark.port.maxRetries', '100')
sc = pyspark.SparkContext.getOrCreate(conf)
conf = sc.getConf()
sc

In [ ]:
# init spark session
spark = SparkSession(sc)

## Loading the data

## Data Processing

### Cleaning metadata
First, let's clean the metadata dataframe:

In [ ]:
# load metadata
raw_metadata = spark.read.load('/datasets/project/metadata', format='com.databricks.spark.csv', header='false', sep='\\t')

In [ ]:
# remove multiple spaces
metadata = raw_metadata.withColumn('_c0', functions.regexp_replace(raw_metadata._c0, '\s+', ' '))
# split into columns
metadata = metadata.withColumn('name', functions.split(metadata._c0, '%')[1])
for (name, index, type_) in [('station_ID',0, 'int'), ('long',1, 'double'), ('lat',2, 'double'), ('height',3, 'int')]:
    metadata = metadata.withColumn(name, functions.split(metadata._c0, ' ')[index].cast(type_))
# remove useless column
metadata = metadata.drop('_c0')
# trim name column to remove left/right blank
metadata = metadata.withColumn('name', functions.trim(metadata.name))

In [ ]:
metadata.show(5)

We will use the SBB data limited around the Zurich area. We will focus on all the stops within 10km of the Zurich train station. Let's get rid of all the stations that are too far away from Zurich:

In [ ]:
metadata.count()

In [ ]:
# coordinates of Zürich main train station
lat_zurich = 47.3782
long_zurich = 8.5402

In [ ]:
# convert to pandas dataframe
pandas_df = metadata.toPandas()
# keep only the stops that are located < 10km from Zurich HB
pandas_df['distance_to_zh'] = pandas_df.apply(lambda x: helpers.distance(x['long'], x['lat'], long_zurich, lat_zurich), axis=1)
pandas_df = pandas_df[pandas_df['distance_to_zh'] < 10]

In [ ]:
pandas_df.distance_to_zh.max()

In [ ]:
# recreate spark dataframe from pandas dataframe
metadata = spark.createDataFrame(pandas_df)

### Cleaning main dataset

In [ ]:
# load full data
raw_df = spark.read.load('/datasets/project/istdaten/*/*', format='csv', header='true', inferSchema='true', sep=';')
# load sample data
# raw_df = spark.read.load('/datasets/project/istdaten/2017/10', format='csv', header='true', inferSchema='true', sep=';')

In [ ]:
# rename the fields german -> english
fields = {
    'BETRIEBSTAG':'date',
    'FAHRT_BEZEICHNER':'trip_id',
    'PRODUKT_ID':'transport_type',
    'LINIEN_ID':'train_id',
    'VERKEHRSMITTEL_TEXT':'train_type',
    'ZUSATZFAHRT_TF':'additional_trip',
    'FAELLT_AUS_TF':'trip_failed',
    'HALTESTELLEN_NAME':'stop_name',
    'BPUIC':'stop_id',
    'ANKUNFTSZEIT':'schedule_arrival',
    'AN_PROGNOSE':'real_arrival',
    'AN_PROGNOSE_STATUS':'arr_forecast_status',
    'ABFAHRTSZEIT':'schedule_dep',
    'AB_PROGNOSE':'real_dep',
    'AB_PROGNOSE_STATUS':'dep_forecast_status',
    'DURCHFAHRT_TF':'no_stop_here'
}

df = raw_df.selectExpr([k + ' as ' + fields[k] for k in fields])
df.show(5)

In [ ]:
# keep only the rows with stops near zurich
df2 = df.where(df.stop_id.isin([int(x) for x in list(pandas_df.station_ID.unique())]))

In [ ]:
# there is still 51'571'541 rows in zurich area
# df2.count()

## Modeling the network

In [ ]:
# df2.where(df2.date == '13.09.2017').orderBy('stop_name').show()

In [ ]:
# df2.select(['trip_id', 'schedule_arrival']).where(df2.trip_id=='85:773:18010-11456-1').orderBy(df2.schedule_arrival, ascending=False).show()

In [ ]:
@functions.udf
def date_choice(arr_time, dep_time):
    '''
    the departure time is null if it's the last stop of the trip
    returns a column that contains the departure time, or the arrival time if it's null
    this will serve to reconstruct the network from the trip's schedules
    '''
    return arr_time if dep_time is None else dep_time

@functions.udf
def stop_type(schedule_departure, schedule_arrival):
    '''
    create a column that tells if a stop is the first/last one of its trip or in the middle
    '''
    if schedule_departure is None:
        return 'last'
    elif schedule_arrival is None:
        return 'first'
    else:
        return 'mid'

In [ ]:
# create a column with the schedule time that will be used to build the network
df2 = df2.withColumn('schedule_time', date_choice(df2.schedule_arrival, df2.schedule_dep))
df2 = df2.withColumn('schedule_time', functions.from_unixtime(functions.unix_timestamp('schedule_time', 'dd.MM.yyyy HH:mm')))

# create a column that tells if a stop is the first/last one of its trip or in the middle
df2 = df2.withColumn('stop_type', stop_type(df2.schedule_dep, df2.schedule_arrival))

In [ ]:
trips = df2.select(['trip_id', 'schedule_time', 'stop_id', 'stop_type', 'schedule_dep', 'schedule_arrival']).orderBy(['trip_id', 'schedule_time'], ascending=[0,0])

In [ ]:
trips.where(trips.trip_id == '85:773:18010-11456-1').show()

In [ ]:
df2.where((df2.schedule_arrival.isNull()) & (df2.trip_id == '85:773:18010-11456-1')).show()